In [1]:
import pandas as pd
import helpers_for_backend as hfb

# We first load our rules, we use lift as our metric in this demo. Can be changed to "lift", "confidence", "zhangs_metric"
extracted_rules = hfb.load_rule_data('./dataset/rules_recipe_scale.csv', metric='lift')

Starting to load rule data
Rule data loaded...

Starting rule extraction...
	 -> Starting to sort rules by lift
	 -> Done sorting rules...
______________________________


In [2]:
# Get a recipe to test
relex_recipes = pd.read_csv(f'./dataset/relex_examples.csv').sample(1, random_state=1010)
relex_recipes.head()

,user_id,event_date,event_type,details,recipe,ks-recipe
162,62e291e6b76e15b147084384,2022-09-04 09:02:09.477058,submit,"{'recipe': ""Recipe for cooking Spaghetti Bolog...",Recipe for cooking Spaghetti Bolognaise:\n\n\n...,"[{'time': 1662281507561, 'character': 'Shift'}..."


In [3]:
# use gensim preprocess_string
from gensim.parsing.preprocessing import preprocess_string
relex_recipes['preprocessed'] = relex_recipes['recipe'].apply(preprocess_string)
relex_recipes.head()

,user_id,event_date,event_type,details,recipe,ks-recipe,preprocessed
162,62e291e6b76e15b147084384,2022-09-04 09:02:09.477058,submit,"{'recipe': ""Recipe for cooking Spaghetti Bolog...",Recipe for cooking Spaghetti Bolognaise:\n\n\n...,"[{'time': 1662281507561, 'character': 'Shift'}...","[recip, cook, spaghetti, bolognais, ingredi, b..."


<div style="width: 100%; display: flex; justify-content: center">
    <h3 style="width: 95%; border: 4px solid #ed8796; color: #ed8796; text-align: center; margin: 0; padding: 10px;">
        Step by step(manual) recipe creation
    </h3>
</div>

In [4]:
# now extract rules, make sure the metric is the same as the one used to extract the rules, since the rules are sorted by the metric
fulfilled_rules, suggestions = hfb.extract_rules(
    recipe=relex_recipes.iloc[0]['preprocessed'], 
    rules=extracted_rules,
    rule_count=5,
    metric='lift',
    )

suggestions

{frozenset({'boil', 'reduc', 'set'}): ("frozenset({'bring', 'asid'})",
  25.285247355772828),
 frozenset({'contain'}): ("frozenset({'store', 'airtight'})",
  24.000669837887127),
 frozenset({'asid', 'boil', 'reduc'}): ("frozenset({'bring', 'set'})",
  23.751010996859893),
 frozenset({'asid', 'boil'}): ("frozenset({'bring', 'set', 'reduc'})",
  20.654708228732744),
 frozenset({'asid', 'oliv'}): ("frozenset({'set', 'oil', 'pepper'})",
  20.61868515054216)}

In [5]:
# Now using the suggestions, we create the prompt
# Note: we give the non-preprocessed recipe to the prompt, since the prompt uses it as the 'original' recipe
prompt = hfb.create_prompt(
    directions=relex_recipes.iloc[0]['recipe'],
    fulfilled_rules=fulfilled_rules,
    suggestions=suggestions,
    )

# Quick look at the prompt
print('\n'.join(prompt.split('\n')[:20]))


    Recipe:
    Recipe for cooking Spaghetti Bolognaise:


Ingredients: 


500g Beef Mince
1 Beef Stock Cube
1 White Onion
1 Carrot
250ml Red Wine
600g Spaghetti
1 tsp Worcestershire Sauce
1 tsp Sugar
1 Container of Passata 





In [6]:
# Now we have our prompt, we prompt chatgpt
response = hfb.prompt_gpt(
    prompt=prompt,
    print_response=False,
    model='gpt-4'
)

In [7]:
# We have this helper function to print the response in a nice format.
hfb._print_response(response)

New recipe:

Ingredients:
- 500g Beef Mince
- 1 Beef Stock Cube
- 1 White Onion
- 1 Carrot
- 250ml Red Wine
- 600g Spaghetti
- 1 tsp Worcestershire Sauce
- 1 tsp Sugar
- 1 Container of Passata
- Olive oil
- Salt and Pepper to taste

Instructions:
1. Dice the onion and carrots into small squares and set aside into two bowls.
2. Add the stock cube to 500ml of hot water and allow it to dissolve.
3. Heat a pan with olive oil and add the onions for a couple of minutes until they are brown, then add the carrots and cook for a further couple of minutes until the carrots begin to soften.
4. Add the mince and cook until it starts to brown.
5. Set another pan on a hob and begin to boil 500ml of water. Add salt to taste.
6. Once the mince has browned, bring the glass of Wine to the pan and leave it to simmer for 5 minutes. This should burn off any excess alcohol.
7. Add the Worcestershire Sauce, Beef stock and Passata to the mince and give it a stir. Leave this to simmer for 25 minutes.
8. Add th

<div style="width: 100%; display: flex; justify-content: center">
    <h3 style="width: 95%; border: 4px solid #ed8796; color: #ed8796; text-align: center; margin: 0; padding: 10px;">
        Using the pipeline function
    </h3>
</div>

In [8]:
# Instead of doing all of this one by one, we can also do the following:
# Note, we still need to load the extracted rules & preprocess the recipe
response_pipeline = hfb.complete_pipeline(
    recipe_tokens=relex_recipes.iloc[0]['preprocessed'],
    recipe_directions=relex_recipes.iloc[0]['recipe'],
    extracted_rules=extracted_rules,
    rule_count=5,
    metric='lift',
    model='gpt-4',
)

In [9]:
hfb._print_response(response_pipeline.new_recipe)

New recipe:

Ingredients:
- 500g Beef Mince
- 1 Beef Stock Cube
- 1 White Onion
- 1 Carrot
- 250ml Red Wine
- 600g Spaghetti
- 1 tsp Worcestershire Sauce
- 1 tsp Sugar
- 1 Container of Passata
- Olive oil
- Salt and pepper to taste

Instructions:
1. Dice the onion and carrots into small squares and set aside into two bowls.
2. Add the stock cube to 500ml of hot water and allow it to dissolve.
3. Heat a pan with olive oil and add the onions for a couple of minutes until they are brown, then add the carrots and cook for a further couple of minutes until the carrots begin to soften.
4. Add the mince and cook until it starts to brown.
5. Set another pan on a hob and begin to boil 500ml of water. Add salt and pepper to taste.
6. Once the mince has browned, bring the glass of Wine to the pan and leave it to simmer for 5 minutes. This should burn off any excess alcohol.
7. Add the Worcestershire Sauce, Beef stock and Passata to the mince and give it a stir. Leave this to simmer for 25 minutes